# ITSG Bug

In [ ]:
import os
os.chdir('/media/abhishek/DATA/pyshbundle/')

import pyshbundle
from tqdm import trange
from datetime import datetime

from pyshbundle.new_io import read_jpl, read_csr, read_tn13, read_tn14, extract_C10_11_replcmnt_coeff, cklm2sc_new
from pyshbundle.new_io import extract_C20_replcmnt_coeff, extract_C30_replcmnt_coeff, replace_zonal_coeff, find_date_in_replacemnt_file
from pyshbundle.new_io import read_itsg

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pyshbundle.reader_replacer_jpl
import pyshbundle.reader_replacer_csr
import pyshbundle.reader_replacer_itsg

from pyshbundle import gshs
from pyshbundle import clm2sc

In [ ]:
def last_4chars(x):
        #print(x[-39:-32])
        return(x[-11:-4])

In [ ]:
def clm2sc_new2(data_mat, lmax):
    sc_mat = np.zeros((lmax+1, 2*lmax + 2))
    dev_sc_mat = np.zeros((lmax+1, 2*lmax + 2))
    clm = data_mat[:, 2]
    slm = data_mat[:, 3]
    clm_std_dev = data_mat[:, 4]
    slm_std_dev = data_mat[:, 5]

    i = 0
    for index1 in range(0,lmax+1, 1):
        for index2 in range(0,index1+1, 1):
            
            sc_mat[index1, lmax-index2] = slm[i]
            sc_mat[index1, lmax+index2+1] = clm[i]

            dev_sc_mat[index1, lmax-index2] = slm_std_dev[i]
            dev_sc_mat[index1, lmax+index2+1] = clm_std_dev[i]

            i = i + 1
    
    sc_mat = np.delete(sc_mat, lmax, 1)
    dev_sc_mat = np.delete(dev_sc_mat, lmax, 1)
    return sc_mat, dev_sc_mat

In [ ]:
tn14_path_itsg = '../pyshbundle/pyshbundle/data/ITSG_TN_files/TN-14_C30_C20_SLR_GSFC.txt'
tn_13_path_itsg = '../pyshbundle/pyshbundle/data/ITSG_TN_files/TN-13_GEOC_CSR_RL06.1.txt'
itsg_folder = '../pyshbundle/sample_input_data/ITSG_input'
itsg_file = '../pyshbundle/sample_input_data/ITSG_input/ITSG-Grace2018_n96_2011-05.gfc'

In [ ]:
data_itsg_mat= np.ndarray((4753, 6))
header = read_itsg(itsg_file)[0]
data_itsg_mat = read_itsg(itsg_file)[1]
dates_itsg = read_itsg(itsg_file)[2]

In [ ]:
type(dates_itsg)

In [ ]:
from datetime import datetime, timedelta
def STR2DON(str_date, source='itsg'):
    if source == "itsg":
        #print("NOTE: ITSG only provides Year-Month, the Date Object Notation will assume it to be the first day of the corresponding month by default! \n")
        return datetime.strptime(str_date, "%Y-%m").date()
    elif source == "csr":
        return datetime.strptime(str_date, "%Y-%m-%d").date()
    elif source == "jpl":
        return datetime.strptime(str_date, "%Y-%m-%d").date()
    else:
        raise ValueError("Enter a valid data source (Processing Centre). Currently only supported for CSR, JPl and ITSG Processing Centers")

In [ ]:
def dates_by_year(date_list: list, source='itsg'):
    if source == 'itsg':
        # accertain the type of list
        if type(date_list[0] == 'str'):
            for i in range(len(date_list)):
                date_list[i] = STR2DON(date_list[i], source='itsg')
        
        else:
            raise ValueError("dates should be a list of strings")
        
        list_of_years = []
        valid_months = set(range(1, 13))
        dates_by_year = {}
        # Group the dates by year
        for date in date_list:
            year = date.year
            if year not in dates_by_year:
                dates_by_year[year] = []
            dates_by_year[year].append(date)

        # Print the dates grouped by year
        return dates_by_year



        pass
    elif source == 'csr':
        pass
    elif source == 'jpl':
        pass
    pass

In [ ]:
import copy
dates_list = copy.deepcopy(dates_start)

In [ ]:
dates_by_year = dates_by_year(dates_list, source='itsg')

In [ ]:
# pretty printing the header info
import pprint

pp = pprint.PrettyPrinter(indent=2, width=2, compact=True)
pp.pprint(dates_by_year)

In [ ]:
dates_by_year[list(dates_by_year.keys())[0]][0].month

In [ ]:
def missing_months(yearwise_dates: dict):
    year_keys = list(yearwise_dates.keys())
    valid_months = set(range(1, 13))
    missing_data = {}

    for year in year_keys:
        
        dates_in_year = yearwise_dates[year]
        months_present = set(date.month for date in dates_in_year)
        missing_months = valid_months - months_present
        
        if len(missing_months) != 0:
            missing_data[year] = list(missing_months)
        else:
            missing_data[year] = 0
    

    return missing_data


In [ ]:
missing_months = missing_months(yearwise_dates=dates_by_year)

In [ ]:
missing_months[2002]

In [ ]:
min(missing_months.keys())

In [ ]:
missing_months

In [ ]:
def timeseries_dataframe(data_dates: list, data: list, missing_months, column_header):
    
    
    

    
    pass

In [ ]:
STR2DON(dates_itsg, source='itsg')

In [ ]:
scmat_itsg, devscmat_itsg = clm2sc_new2(data_itsg_mat, lmax=96)

In [ ]:
type(data_itsg_mat)

In [ ]:
from pyshbundle import sc2cs

In [ ]:
a = sc2cs.sc2cs(clm2sc_new2(data_itsg_mat, 96)[0])

In [ ]:
plt.imshow(np.ma.log10(np.absolute(a)))

In [ ]:
type(scmat_itsg)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
degree_mean = []
for i in range(97):
    desparsed = np.absolute(de_sparse(scmat_itsg[i, :]))
    degree_mean.append(np.mean(desparsed))
    x = i * np.ones((len(desparsed)))

    ax.scatter(x, np.log10(desparsed), color='blue', marker='+', alpha=0.5)

ax.plot(range(97), np.log10(degree_mean), 'k', linestyle='--', label="Mean per degree")

x_vec = np.arange(0, 96+1, 6)

plt.ylim([-11, -5])
plt.xlim([0, 97])
plt.ylabel("coefficients log10")
plt.xlabel("Degree (n)")
plt.xticks(x_vec)
plt.title("Absolute value of the normalized (Unreplaced) GRACE ITSG coeficients")
plt.legend()
plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(20, 10))
degree_mean = []
for i in range(97):
    desparsed = np.absolute(de_sparse(rep_sc[i, :]))
    degree_mean.append(np.mean(desparsed))
    x = i * np.ones((len(desparsed)))

    ax.scatter(x, np.log10(desparsed), color='blue', marker='+', alpha=0.5)

ax.plot(range(97), np.log10(degree_mean), 'k', linestyle='--', label="Mean per degree")

x_vec = np.arange(0, 96+1, 6)

plt.ylim([-11, -5])
plt.xlim([0, 97])
plt.ylabel("coefficients log10")
plt.xlabel("Degree (n)")
plt.xticks(x_vec)
plt.title("Absolute value of the normalized (Replaced) GRACE ITSG coeficients")
plt.legend()
plt.show()

In [ ]:
import scipy.sparse as sps

a = sps.csr_matrix(scmat_itsg)


In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))
plt.imshow(np.ma.log10(abs(scmat_itsg)), cmap="Spectral_r")
plt.colorbar()
plt.title("Raw Data - New Implementaion")
plt.show()

In [ ]:
scmat_itsg[0:5, 91:102]

In [ ]:
# Read replacemnt file
data_tn13_itsg = read_tn13(tn_13_path_itsg)
data_tn14_itsg = read_tn14(tn14_path_itsg)

In [ ]:
data_tn13_itsg[0:5, :]

In [ ]:
data_tn14_itsg[0:5, :]

# Let's Observe the Replacemnt Procedure

In [ ]:
replaced_data_mat = replace_zonal_coeff(data_itsg_mat, 'itsg', 96, data_tn13_itsg, data_tn14_itsg, dates_itsg, epoch_end=None)

In [ ]:
rep_sc, rep_dev = clm2sc_new2(replaced_data_mat, lmax=96)

In [ ]:
rep_sc[0:5, 91:102]

In [ ]:
epoch_begin = datetime.strptime(dates_itsg, '%Y-%m').date()
tn_13_idx = find_date_in_replacemnt_file(data_tn13_itsg, "tn-13", "itsg", epoch_begin)
print(f"The TN-13 indices found to match : {tn_13_idx}")

In [ ]:
# This is the TN-14 Replacement file data from the matrix

epoch_begin = datetime.strptime(dates_itsg, '%Y-%m').date()
tn_14_idx = find_date_in_replacemnt_file(data_tn14_itsg, "tn-14", "itsg", epoch_begin)
print(f"The TN-14 indices found to match : {tn_14_idx}")

## Vivek's Approach

In [ ]:
itsg_clm, dates_start, no_of_files = pyshbundle.reader_replacer_itsg.reader_replacer_itsg(itsg_folder, tn14_path_itsg, tn_13_path_itsg)

In [ ]:
itsg_sc, itsg_dev_sc = pyshbundle.clm2sc.clm2sc(itsg_clm)

In [ ]:
itsg_sc[105, 0:5, 91:102]

In [ ]:
np.sum(abs(itsg_sc[105, :] - rep_sc))

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))
plt.imshow(np.ma.log10(abs((itsg_sc[105, :] - rep_sc))), )
plt.colorbar()
plt.title("Replaced Data - Difference")
plt.show()

In [ ]:
# load all the itsg files
files = os.listdir(itsg_folder)
files = sorted(files, key = last_4chars)

data_itsg_mat = np.ndarray((len(files), 4753, 6))
dates_itsg = []
for i in trange(len(files)):
    file_path = itsg_folder + '\\' + files[i]
    header = read_itsg(file_path)[0]
    data_itsg_mat[i, :, :] = read_itsg(file_path)[1]
    dates_itsg.append(read_itsg(file_path)[2])

In [ ]:
header

In [ ]:
type(read_itsg(file_path)[2])

In [ ]:
#replace_zonal_coeff(data_mat, data_tn13, data_tn14, epoch_begin, epoch_end)
# Replace zonal coeff
replaced_data_mat_itsg = np.ndarray((len(files), 4753, 6))

unreplaced_idx = set()
for i in trange(len(files)):
    try:
        replaced_data_mat_itsg[i, :, :] = replace_zonal_coeff(data_itsg_mat[i, :, :], 'itsg', 96, data_tn13_itsg, data_tn14_itsg, dates_itsg[i], epoch_end=None)
        print("\n")
    except IndexError:
        print(f"Replacement data for File Index {i} - {dates_itsg[i]} could not be found.... procedding to next file")
        unreplaced_idx.add(i)

unreplaced_idx = list(unreplaced_idx)

In [ ]:
scmat_deck_itsg = np.ndarray((len(files), 97, 193))
devscmat_deck_itsg = np.ndarray((len(files), 97, 193))

for i in trange(196):
    if i not in unreplaced_idx:
        scmat_deck_itsg[i, :, :], devscmat_deck_itsg[i, :, :] = clm2sc_new2(replaced_data_mat_itsg[i, :, :], lmax=96)

In [ ]:
scmat_diff_itsg = np.ndarray((len(files), 97, 193))
devscmat_diff_itsg = np.ndarray((len(files), 97, 193))

for i in range(193):
    scmat_diff_itsg[i, :, :] = scmat_deck_itsg[i, :, :] - itsg_sc[i, :, :]
    devscmat_diff_itsg[i, :, :] = devscmat_deck_itsg[i, :, :] - itsg_dev_sc[i, :, :]

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))
plt.imshow(np.ma.log10(abs(devscmat_diff_itsg[0, :, :])))
plt.colorbar()
plt.title("Replaced Data - Difference")
plt.show()